In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
#https://stackoverflow.com/questions/19726663/how-to-save-the-pandas-dataframe-series-data-as-a-figure
import six



In [2]:
def add_latestwk(df, stats):
    for stat in stats:
        df['%s (May 21st)' % stat] = df.filter(regex=("%s \(w.*" % stat)).apply(lambda x: x.dropna().get(-1, None), axis=1)
    return df


tbpath = "/Users/aavattikutis/Box/covid-19_share/fit10tablesnew/"
df = pd.read_csv(tbpath + "fit_table_reweighted.csv") 

df = add_latestwk(df, ['Rt', 'car', 'ifr'])

# print(df)

productpath = '../../results/'

In [3]:
rois = list(df.roi.unique())

roi_us = np.sort([i for i in rois if i[:2]=='US'])
roi_other = np.sort([i for i in rois if i[:2]!='US'])

rois = list(roi_other) + list(roi_us)

rois.remove('US')
# rois.remove('Gambia')
rois.remove('AA_Global')

In [4]:
theta_ = ["R0","car (week 0)", "ifr (week 0)","Rt (May 21st)",
          "car (May 21st)", "ifr (May 21st)" ]

# round_ =[1,2,4,1,2,4]
# label_ = {}
# label_["R0"] = "R0"
# label_["car (week 0)"] = "CAR (week 0)"
# label_["ifr (week 0)"] = "IFR (week 0)"
# label_["Rlast"] = "R current"
# label_["extra_std"] = "Extra Variability"

def afun1(x):
    return '%s' % float('%.1g' %x)

def afun2(x):
    return '%s' % float('%.2g' %x)

dfreport = pd.DataFrame(columns=
                        ['Region']+
                        ["R0 (CI)"]+
                        ["ci1"]+
                        ["CARt (week 0) (CI)"]+
                        ["ci2"]+
                        ["IFRt (week 0) (CI)"]+
                        ["ci3"]+
                        ['Rt(May 21st, 2020) (CI)']+
                        ["ci4"]+
                        ['CARt(May 21st, 2020) (CI)']+
                        ["ci5"]+
                        ['IFRt(May 21st, 2020) (CI)']+
                        ["ci6"]+
                        ["delta weeks"])



k = -1
for roi in rois:
    k += 1
    x = [roi]
    for i in range(len(theta_)):
        theta = theta_[i]
#         print(roi)
#         print(theta)
        mu = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
        lb = df.loc[(df.roi==roi)&(df['quantile']=='0.025'),theta].values[0]
        ub = df.loc[(df.roi==roi)&(df['quantile']=='0.975'),theta].values[0]
        if theta[0] == 'R':
            x += [afun2(mu)]+["("+afun2(lb)+", "+afun2(ub)+")"]
        else:
            x += [afun1(mu)]+["("+afun1(lb)+", "+afun1(ub)+")"]
#         print(x)
    for week in np.arange(11,0,-1):
        theta = 'ifr (week '+str(week)+')'
        x2 = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
        if np.isfinite(x2):
            break
    x += [week]
#     print(x)
    try:
        dfreport.loc[k] = x
    except:
        pass
dfreport.to_csv(productpath + "theta_summary.csv",sep='\t',index=False)


In [5]:
# def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
#                      header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
#                      bbox=[0, 0, 1, 1], header_columns=0,
#                      ax=None, **kwargs):
#     if ax is None:
#         size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
#         fig, ax = plt.subplots(figsize=size)
#         ax.axis('off')

#     mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

#     mpl_table.auto_set_font_size(False)
#     mpl_table.set_fontsize(font_size)

#     for k, cell in six.iteritems(mpl_table._cells):
#         cell.set_edgecolor(edge_color)
#         if k[0] == 0 or k[1] < header_columns:
#             cell.set_text_props(weight='bold', color='w')
#             cell.set_facecolor(header_color)
#         else:
#             cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
#     return ax
#     render_mpl_table(dfreport, header_columns=0, col_width=8.0)
#     plt.savefig("../postfit_derivatives/"+theta+"_summary.png")
#     plt.clf()